In [ ]:
!pip install transformers --quiet

In [1]:
!id="18E24-G2Xq-y5HfM6ZatUB_8WKL_r6HOV"; conf=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$id -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p'); wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$conf&id=$id" -O pubhealth.csv && rm -rf /tmp/cookies.txt

--2022-08-25 12:33:21--  https://docs.google.com/uc?export=download&confirm=&id=18E24-G2Xq-y5HfM6ZatUB_8WKL_r6HOV
Resolving docs.google.com (docs.google.com)... 142.250.203.206, 2a00:1450:401b:810::200e
Connecting to docs.google.com (docs.google.com)|142.250.203.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-34-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hu3hf9fb264l5mne6gmhhtkla9f5dc4e/1661423550000/05536367513713019519/*/18E24-G2Xq-y5HfM6ZatUB_8WKL_r6HOV?e=download&uuid=a2cba134-b25d-4da1-a1bb-a764bceac670 [following]
--2022-08-25 12:33:24--  https://doc-0k-34-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hu3hf9fb264l5mne6gmhhtkla9f5dc4e/1661423550000/05536367513713019519/*/18E24-G2Xq-y5HfM6ZatUB_8WKL_r6HOV?e=download&uuid=a2cba134-b25d-4da1-a1bb-a764bceac670
Resolving doc-0k-34-docs.googleusercontent.com (doc-0k-34-docs.googleusercontent.com)... 142.250.75.1, 2a00:1450:40

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import RepeatedStratifiedKFold

data = pd.read_csv("../../raw/nela-gt-2020.csv",sep=",")
print("labels\n",data["label"].value_counts())
print("shape  \n",data.shape)

bootstrap_size = 0

if bootstrap_size != 0:
    bootstrap_factor = bootstrap_size / data.shape[0]
    bootstrap = np.random.uniform(size=data.shape[0]) < bootstrap_factor
    data = data.iloc[bootstrap]

data["text"] = [ str(v) for v in data["text"]]
X, y = data["text"].values, data["label"].values

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

labels
 0    494808
1    262288
Name: label, dtype: int64
shape  
 (757096, 9)


100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 38.14it/s]

shapes X (757096,) y (757096,)


In [3]:
data.columns

Index(['Unnamed: 0', 'source', 'title', 'text', 'author', 'url',
       'published_utc', 'collection_utc', 'label'],
      dtype='object')

In [2]:
import pickle

# SAVE
with open('nela-gt-X.pickle', 'wb') as handle:
    pickle.dump(X, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('nela-gt-y.pickle', 'wb') as handle:
    pickle.dump(y, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [1]:
import pickle

# load
# with open('nela-gt-X.pickle', 'rb') as handle:
#     X=pickle.load(handle)
    
with open('../../pickles/products/nela-gt-X.pickle', 'rb') as handle:
    X=pickle.load(handle)    
    
with open('../../pickles/products/nela-gt-y.pickle', 'rb') as handle:
    y=pickle.load(handle)
    
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import RepeatedStratifiedKFold

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X", X.shape,y.shape)

100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 37.97it/s]

shapes X (757096,) (757096,)


In [4]:
import transformers
transformers.logging.set_verbosity_error()
from torch.utils.data import DataLoader 

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import TFDistilBertModel, DistilBertTokenizerFast

from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np

from scipy.sparse import csr_matrix
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device:", device)

if "disilbert_model" not in locals():
    disilbert_tokenizer =  AutoTokenizer.from_pretrained("distilbert-base-uncased")
    disilbert_model = AutoModel.from_pretrained("distilbert-base-uncased")
    handle = disilbert_model.to(device)


class BERTEmbeddings(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer =  disilbert_tokenizer
        self.model = disilbert_tokenizer
        self.max_length = 256
        self.model_name = disilbert_model

    def fit(self, X=None, y=None):
        pass
    
    def encode(self, txt):
        return self.tokenizer(txt, max_length=self.max_length, 
                              truncation=True, padding=True, return_tensors="pt")

    def transform(self, X=None):
        dataloader = DataLoader(X, batch_size=4, shuffle=False)
        allembeds = []
        for batch in tqdm(dataloader):
            batchenc = disilbert_tokenizer(batch, max_length=256, truncation=True, padding=True, return_tensors="pt")
            input_ids = batchenc['input_ids'].to(device)
            attention_mask = batchenc['attention_mask'].to(device)
            batchout = disilbert_model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
            embeds = [vec[0].cpu().detach().numpy() for vec in batchout[1][-1]]
            allembeds.extend(embeds)
        return csr_matrix(allembeds)


class BertHead(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.head = LogisticRegression(class_weight='auto', max_iter=10000)

    def fit(self, X=None, y=None):
        self.head.fit(X, y)


    def transform(self, X=None):
        pass
    
    def predict(self, X=None):    
        return self.head.predict(X)
    
    def predict_proba(self, X=None):    
        return self.head.predict_proba(X)



using device: cuda


In [4]:
bert = BERTEmbeddings()
X_dstil_numpy = bert.transform(X).toarray()

100%|█████████████████████████████████████| 47319/47319 [24:03<00:00, 32.78it/s]


In [3]:
import pickle

# SAVE
with open('pickles/bertcls_embeddings/nela-gt_BERTEmbeddings.pickle', 'wb') as handle:
    pickle.dump(X_dstil_numpy, handle, protocol=pickle.HIGHEST_PROTOCOL)

FileNotFoundError: [Errno 2] No such file or directory: 'pickles/bertcls_embeddings/nela-gt_BERTEmbeddings.pickle'

In [2]:
import pickle

with open('../../pickles/products/nela-gt-y.pickle', 'rb') as handle:
    y=pickle.load(handle)

# load
with open('../../pickles/bertcls_embeddings/nela-gt_BERTEmbeddings.pickle', 'rb') as handle:
    X_dstil_numpy = pickle.load(handle)
    
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import RepeatedStratifiedKFold

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X_dstil_numpy, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X_dstil_numpy.shape,"y", y.shape)

100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 35.78it/s]

shapes X (757096, 768) y (757096,)


In [5]:
from sklearn.metrics import log_loss,accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.metrics import geometric_mean_score
from IPython.display import clear_output

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def log_los_nonorm(x,y,average=False):
    return log_loss(x,y,normalize=True)

scores = {
    'Accuracy': {'func': accuracy_score},
    'Balanced Accuracy': {'func': balanced_accuracy_score},
    'F1': {'func': f1_score},
    'Precision': {'func': precision_score},
    'Recall': {'func': recall_score},
    'Log-likelihood': {'func': log_los_nonorm},
    #'G-mean': {'func': geometric_mean_score}
}

for score_name, score_dict in scores.items():
    scores[score_name]["list"] = []
    scores[score_name]["lab"] = []


for fold,j in enumerate(foldids):
    train = foldids[fold][1]
    test = foldids[fold][2]
    xin, yin = X_dstil_numpy[train], np.array(y[train])
    cls = BertHead()
    
    cls.fit(xin, yin)
    y_pred = cls.predict(X_dstil_numpy[test])
    y_pred_proba = cls.predict_proba(X_dstil_numpy[test])
    
    for score_name, score_dict in scores.items():
        if score_name == "Log-likelihood":
            scorval=score_dict['func'](y[test], y_pred_proba)
            score_dict['list'].append(scorval)
            print(score_name, scorval)
        elif score_name in ["F1","Precision","Recall"]:
            scorvaln = score_dict['func'](y[test], y_pred, average=None)
            score_dict['lab'].append(scorvaln)
            scorval = score_dict['func'](y[test], y_pred, average="weighted")
            score_dict['list'].append(scorval)
            print(score_name, scorval, scorvaln)  
        else:
            scorval=score_dict['func'](y[test], y_pred)
            score_dict['list'].append(scorval)
            print(score_name, scorval)
    print()

Accuracy 0.8267115398839777
Balanced Accuracy 0.7971207305688068
F1 0.8244339959346526 [0.87079225 0.73697885]
Precision 0.8242570074839703 [0.84923739 0.77713136]
Recall 0.8267115398839777 [0.89346979 0.70077167]
Log-likelihood 0.3897019096148514

Accuracy 0.8275198917970773
Balanced Accuracy 0.7978627738341977
F1 0.8252196411380559 [0.87143752 0.73802932]
Precision 0.8250791442440715 [0.84959955 0.77882124]
Recall 0.8275198917970773 [0.89442774 0.70129781]
Log-likelihood 0.38967154333319703

Accuracy 0.826801356763211
Balanced Accuracy 0.7968597865578693
F1 0.8244447360276969 [0.87096139 0.73669076]
Precision 0.8243251113127108 [0.84876406 0.77822088]
Recall 0.826801356763211 [0.89435094 0.69936863]
Log-likelihood 0.3902715987716882

Accuracy 0.8280006762682671
Balanced Accuracy 0.7986910388460167
F1 0.8257724253162558 [0.87171202 0.73910709]
Precision 0.8255948062962692 [0.85039558 0.77880799]
Recall 0.8280006762682671 [0.89412459 0.70325749]
Log-likelihood 0.38914869114841866

Accu

In [6]:
numlabels = scores["F1"]["lab"][0].shape[0]
scores["F1"]["lab"][0].shape[0] 
head = "| %-20s | %-12s |" +  numlabels * " %-10s |" 
headv = ["Score", "Average"]
headv.extend(["Kat_"+str(i+1) for i in range(numlabels)])
row=head % tuple(headv)
#print("+"*len(row))
print(row)
#print("+"*len(row))

for score_name, score_dict in sorted(scores.items()) :
    if score_name == "Log-likelihood":
        headv = [score_name, np.mean(score_dict['list'])*1, np.std(score_dict['list'])*1]
    else:
        headv = [score_name, np.mean(score_dict['list'])*100, np.std(score_dict['list'])*100]
    
    for i in range(numlabels):
        if score_name == "Log-likelihood":
            head = "| %-20s | %3.2f ± %3.3f |" + numlabels * " %-10s |" 
            headv.append("-")
        elif score_name in ["F1","Precision","Recall"]:
            head = "| %-20s | %4.1f ± %3.3f |" + numlabels* " %4.1f ± %3.1f |"
            vals = [v[i] for v in scores[score_name]["lab"]]
            headv.append(np.mean(vals)*100)
            headv.append(np.std(vals)*100)
        else:
            head = "| %-20s | %4.1f ± %3.3f |" + numlabels * " %-10s |" 
            headv.append("-")
    print(head % tuple(headv))
    
#print("+"*len(row))

| Score                | Average      | Kat_1      | Kat_2      |
| Accuracy             | 82.7 ± 0.038 | -          | -          |
| Balanced Accuracy    | 79.8 ± 0.055 | -          | -          |
| F1                   | 82.5 ± 0.041 | 87.1 ± 0.0 | 73.8 ± 0.1 |
| Log-likelihood       | 0.39 ± 0.001 | -          | -          |
| Precision            | 82.5 ± 0.040 | 84.9 ± 0.1 | 77.8 ± 0.1 |
| Recall               | 82.7 ± 0.038 | 89.4 ± 0.1 | 70.1 ± 0.1 |
